In [1]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import sys
sys.path.append('/mnt/0A2AAC152AABFBB7/CGE/luxgiant-dstream')
from luxgiant_dstream.prep_ds import PrepDS
from luxgiant_dstream.gwas_fixed import GWASfixed
from luxgiant_dstream.gwas_random import GWASrandom

In [3]:
params = json.load(open("//home/luis/data/testDownStream/configFiles/parameters.JSON"))
paths = json.load(open("//home/luis/data/testDownStream/configFiles/paths_local.JSON"))

In [ ]:
prep = PrepDS(
    input_path=paths['input_directory'],
    input_name=paths['input_prefix'],
    output_path=paths['output_directory'],
    output_name=paths['output_prefix'],
    config_dict=params,
    dependables_path=paths['dependables_directory'],
)

In [ ]:
prep_steps = {
    'ld_prune': prep.exclude_high_ld_hla,
    'pca'     : prep.pca_decomposition
}

for step in prep_steps.keys():
    prep_steps[step]()

In [ ]:
gwas_f = GWASfixed(
    input_path = paths['input_directory'],
    input_name = paths['input_prefix'],
    output_path= paths['output_directory'],
    output_name= paths['output_prefix'],
    config_dict= params,
    preps_path = os.path.join(paths['output_directory'], 'preparatory'),
)

In [ ]:
gwas_f_steps = {
    'train_model': gwas_f.fixed_model_association_analysis,
    'top_hits'   : gwas_f.get_top_hits,
    'annotate'   : gwas_f.annotate_top_hits,
    'plots'      : gwas_f.plot_drawings
}

for step in gwas_f_steps.keys():
    gwas_f_steps[step]()

In [ ]:
gwas_r = GWASrandom(
    input_path = paths['input_directory'],
    input_name = paths['input_prefix'],
    output_path= paths['output_directory'],
    output_name= paths['output_prefix'],
    config_dict= params,
    preps_path = os.path.join(paths['output_directory'], 'preparatory'),
)

In [ ]:
gwas_r_steps = {
    'preparatory': gwas_r.prepare_aux_files,
    'grm'        : gwas_r.compute_grm,
    'random'     : gwas_r.run_gwas_random,
    'top_hits'   : gwas_r.get_top_hits,
    'annotate'   : gwas_r.annotate_top_hits,
    'plots'      : gwas_r.plot_drawings
}
for step in gwas_r_steps.keys():
    gwas_r_steps[step]()